In [1]:
import pandas as pd
import re # Регулярные выражения.
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import pymorphy2 # Морфологический анализатор.
import datetime # Новости будем перебирать по дате.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.
import numpy

In [2]:
train = pd.read_csv('data/train.csv', sep='\t', index_col='id')
test = pd.read_csv('data/test.csv', sep='\t', index_col='id')
other = pd.read_csv('data/other.csv', sep='\t')

In [3]:
train.head()

,name,description,target
id,,,
0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [4]:
test.head()

,name,description
id,,
200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


In [5]:
other.head()

,name,description
0,Специалист научно-производственного отдела,"<p>Образование - Среднее специальное, высшее</..."
1,Оператор по отгрузке товара в 1С (ТЗ),<p><strong>Обязанности:</strong></p> <ul> <li>...
2,Менеджер по персоналу,<strong>Обязанности:</strong> <ul> <li>Подбор ...
3,Ведущий бухгалтер по учёту заработной платы,<strong>Обязанности:</strong> <ul> <li> <p>Нач...
4,Инженер-расчетчик в строительный отдел,<p><strong>Требования:</strong></p> <ul> <li>у...


In [11]:
set(train[train['target']==1].name)

{'Консультант в магазин Дьюти Фри (Шереметьево 2)',
 'Администратор (ЖК Юдино)',
 'Торговый представитель в г. ТАРА',
 'Менеджер торгового сектора (Планета)',
 'Менеджер по продаже сервиса',
 'Начальник отдела офисных продаж',
 'Продавец-кассир ТЦ "Южный"',
 'Продавец-консультант TAKE AWAY (ТРЦ "Райкин Плаза")',
 'Старший смены (ТЦ Академический)',
 'Торговый представитель (Беляевское направление)',
 'Продавец-консультант (ТЦ Пушкинский)',
 'Продавец-консультант по продаже крупной бытовой техники',
 'Директор магазина (г. Заозерный)',
 'Руководитель магазина красоты и заботы',
 'Продавец-консультант в магазин женской одежды (ТЦ Коломенский)',
 'Торговый представитель г. Казань',
 'Менеджер отдела (Киевское шоссе)',
 'Специалист по продажам в Интернет-Магазин (Саратов)',
 'Торговый представитель, BC',
 'IT Sales Manager',
 'Торговый представитель (г. Подольск, Московский, Видное, Домодедово)',
 'Продавець-консультант (ТРЦ "Ocean Plaza")',
 'Торговый представитель (канал HoReCa)',
 'Руко

In [95]:
extensionsToCheck = ['торг', 'прода', 'магаз', 'касс', 'розни', 'мерч', 'товар', 'фасо', 'склад', 'автоколонн', 'лпу', 'кадр', 'дистриб', 'тендер', 'b2']

train['isProd'] = train.apply(lambda x: 1 if any(ext in (x['name'] + x['description']).lower() for ext in extensionsToCheck) else 0, axis=1)
test['isProd'] = test.apply(lambda x: 1 if any(ext in (x['name'] + x['description']).lower() for ext in extensionsToCheck) else 0, axis=1)

In [59]:
x = 'Продавец'
any(ext in x.lower() for ext in extensionsToCheck)

True

In [60]:
train[(train.target != train.isProd) & (train.target==1)]

,name,description,target,isProd
id,,,,
167,Менеджер по привлечению клиентов малого и сред...,<p><strong>От вас требуется:</strong></p> <ul>...,1,0
256,Телемаркетолог,<strong>Обязанности:</strong> <ul> <li>прием з...,1,0
276,Мойщица / Мойщик (ТК Ривер Хаус),<strong>Обязанности:</strong> <ul> <li>Уборка ...,1,0
311,Оператор call-центра,<strong>Обязанности:</strong> <ul> <li>Холодны...,1,0
332,Ночной менеджер,<p>Требования:</p> Опыт работы в аналогичной д...,1,0
367,Менеджер по привлечению клиентов малого и сред...,<p><strong>От вас требуется:</strong></p> <ul>...,1,0
537,Региональный агент,<p><strong>Московский Международный Университе...,1,0
646,Специалист по работе с клиентами,"<p>Если вы ориентированы на результат, желаете...",1,0
697,Менеджер по работе с клиентами,<p><b><br />Салоны автомобилей с пробегом— мол...,1,0


In [61]:
train[(train.target != train.isProd) & (train.target==0)]

,name,description,target,isProd
id,,,,
9,Менеджер по работе с ключевыми клиентами (POSM),<p><strong>Обязанности</strong>:</p> <ul> <li>...,0,1
11,"Продавец-кассир (г Орск, ул Станиславского, 13)","<div>Для каждого, кто хочет работать и зарабат...",0,1
13,"Продавец-кассир (г Истра, ул 9 Гвардейской Див...","<div>Для каждого, кто хочет работать и зарабат...",0,1
15,Программист 1С,<strong>Обязанности:</strong> <ul> <li>Програм...,0,1
16,"Директор магазина (г Кемерово, ул Свободы, 12)",<div>Ведущая федеральная розничная сеть магази...,0,1
20,PR-менеджер,<strong>Обязанности:</strong> <ul> <li>Обеспеч...,0,1
22,"Продавец-кассир (г Санкт-Петербург, ул Шаврова...","<div>Для каждого, кто хочет работать и зарабат...",0,1
24,"Продавец-кассир (г Ставрополь, ул Пирогова, 44)","<div>Для каждого, кто хочет работать и зарабат...",0,1
45,Маркетолог (Химки),<p><strong>Обязанности:</strong></p> <ul> <li>...,0,1


In [66]:
train[train['target']==1]['description'][0]

'<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на постоянную работу руководителя секции (хозтовары).</strong></p> <p><strong>Обязанности: </strong></p> <p> </p> <ul> <li>организация эффективного и культурного обслуживания покупателей</li> <li>организация приемки-сдачи товаров на склад</li> <li>консультация покупателей по вопросам, касающимся оказываемых услуг</li> <li>контроль своевременной подачи товаров в торговую секцию</li> <li>проверка качества, сроков годности ТМЦ, наличие маркировок, ценников на товарах</li> <li>организация бесперебойной работы товарной секции и участка склада, закрепленного за ней</li> <li>контроль за сохранностью товаров, торгового оборудования и прочих материальных ценностей</li> <li>инвентаризация товаров</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>знание товарной группы</li> <li>знание правил приемки товара</li> <li>опыт работы в розничной торговле в качестве продавца 6 р. / заместителя заведующего секцией/отделом</li> <li>отве

In [67]:
train[train['target']==1]['description'][100]

'<p><strong>Компания </strong><strong>SKS</strong> <strong>Partners</strong> <strong>объявляет о вакансии: Руководитель отдела продаж. </strong></p> <p> </p> <p><strong><em>Наша компания занимает лидирующие позиции по производству и продаже игрового оборудования в СНГ. Мы занимаемся комплектацией торгово/семейно-развлекательных центров игровым оборудованием под ключ, автоматизацией бизнес- процессов различных сфер (торговые центры, парки аттракционов, рестораны, кинотеатры и др). Наш сайт - http://skspartners.kz/</em></strong></p> <p> </p> <p><strong>Основные задачи Руководителя отдела продаж</strong></p> <ul> <li>Руководит сбытом продукции компании, разрабатывает ценовую и скидочную политику.</li> <li>Организует и контролирует работу менеджеров по продажам.</li> <li>Координирует разработку перспективных и текущих планов сбыта продукции.</li> <li>Организует работу по ведении, анализу и систематизации клиентской базы.</li> <li>Контролирует состояние дебиторской и кредиторской задолженно

In [73]:
train[train['target']==1]['description'][4]

'<p><strong>Компания Палладиум Стандарт - признанный лидер продаж на рынке дверной фурнитуры приглашает на работу энергичных сотрудников на открытые вакансии мерчендайзер/продавец-консультант для работы в магазине партнерских сетей , отдел замочно - скобяных изделий:</strong></p> <p> </p> <p>Леруа Мерлен Косино (ул. Святоозерская, 1А, ТРЦ Косино Парк).</p> <p>ОБИ Варшавка, (Варшавское ш., 97, ТЦ Ритейл Парк)</p> <p> </p> <p><strong>Обязанности:</strong></p> <ul> <li>Контроль поставки и приемки товара на складе в сети (контроль вывоза товара в зал)</li> <li>Выкладка продукции на полки (контроль ценников, товарного вида, маркировки и т.п.)</li> <li>Консультация покупателей</li> <li>Составление заказов исходя из потребностей и остатков на складе</li> <li>Возвраты (сбор неликвидного товара), принятие мер по минимизации возврата (например, расширенная выкладка или корзина)</li> <li>Рекомендации по ассортименту и улучшению работы, анализ конкурентов</li> <li>Отчетность</li> </ul> <p> </p> <p

In [71]:
train[train['target']==1]['description'][50]

'<p><strong>ПАО «Мобильные ТелеСистемы» (МТС)</strong> является ведущим телекоммуникационным оператором в России и странах СНГ и входит в ТОП-10 крупнейших мобильных операторов мира по размеру базы абонентов.</p> <p><strong>Розничная сеть МТС (АО «РТК»)</strong> - первая в России мобильная розничная сеть, работающая под брендом оператора и 100%-я дочерняя структура ПАО «МТС».</p> <p>В Компании «Розничная сеть МТС» <strong>открыт конкурс на вакансию</strong> «Аналитик» в группы аналитики и внутренней отчетности.</p> <p><strong>В Ваши обязанности будут входить:</strong></p> <ul> <li>Ведение авансовых отчетов Директора Департамента продаж (работа в OEBS);</li> <li>Расчет победителей Мотивационных программ;</li> <li>Сбор и анализ отчетности от ФГ Административной поддержки;</li> <li>Формирование, согласование и контроль бюджета командировочных расходов ALA и AFA;</li> <li>Формирование перечня Индивидуальных вознаграждений работников ОП;</li> <li>Организация совещаний.</li> </ul> <p><strong

In [75]:
train[train['target']==1]['description'][501]

'<strong>Обязанности:</strong> <ul> <li> <p>Крупной розничной сети магазинов продуктов питания требуются торговые представители (на прямые продажи).</p> <p>Прием и выкладка продукции,</p> <p>Продажа продукции,</p> <p>Сдача инкассации, ведение ежедневной отчетности,</p> <p>Проведение еженедельной инвентаризации.</p> </li> </ul> <strong>Требования:</strong> <ul> <li> <p>Рассматриваем кандидатов как с опытом (продавец, продавец-кассир, торговый представитель и др.), так и без опыта работы.</p> <p>Ответственное отношение к работе</p> <p>Вежливость</p> </li> </ul> <strong>Условия:</strong> <ul> <li> <p>Мы предлагаем:</p> <p>График работы: 7/7 с 8.00 до 20.00.</p> <p>Достойная заработная плата, своевременные выплаты.</p> <p>Место работы: г. Лакинск</p> <p>Компания предоставляет бесплатное обучение, перспективу карьерного роста.</p> <p>Торговая точка оснащена современным оборудованием.</p> <p>Официальное оформление.</p> <p>Полный социальный пакет (оплачиваемый больничный лист, оплачиваемый еж

In [78]:
train[train['target']==1]['description'][611]

'<p><strong>Обязанности:</strong></p> <p>• Увеличение объёмов продаж на закрепленной территории;<br />• Увеличение представленности продукции компании в секторе;<br />• Контроль дебиторской задолженности;<br />• Мерчендайзинг.</p> <p> </p> <p><strong>Требования:</strong></p> <p>• Успешный опыт работы в продажах (продукты питания) от 1 года;<br />• Желание работать на результат и зарабатывать;<br />• Личный автомобиль и права категории В;<br />• Ответственность, нацеленность на результат, коммуникабельность, гибкость мышления</p> <p> </p> <p><strong>Условия:</strong></p> <p>• Достойная оплата труда, четкая система мотивации;<br />• Работа в крупной федеральной компании-производителе колбасных изделий и деликатесов торговых марок «Генеральские колбасы», «Дым Дымыч» и «Россиянка»<br />• Компенсация мобильной связи, топ. карта;<br />• Оформление согласно ТК РФ;<br />• Обучение навыкам и техникам продаж;<br />• Возможность карьерного роста.</p> <p> </p>'

In [79]:
train[train['target']==1]['description'][111]

'<p>Месторасположение: Ярославль (ул. Базовая), Россия</p> <p>Территория: Ярославль, Иваново, Кострома, Вологда, Череповец</p> <p><strong>Назначение позиции:</strong></p> <ul> <li> <p>Продажа технических газов и промышленных решений (ГАЗ+Применение+Услуги) широкому кругу покупателей в различных отраслях промышленности внутри определенной географической области.</p> </li> <li> <p>Поддержание и развитие существующей базы клиентов, поддержание и развитие прибыли внутри собственной территории и достижение удовлетворенности покупателей.</p> </li> </ul> <p><strong>Основные обязанности:</strong></p> <p>Выполнение регионального плана продаж на своей территории:</p> <ul> <li> <p>Обеспечение продаж на своей территории в соответствии с целями</p> </li> <li> <p>Анализ продаж и определение дальнейших шагов</p> </li> <li> <p>Анализ деятельности конкурентов</p> </li> <li> <p>Подготовка отчетов по своей территории</p> </li> <li> <p>Поддержка отдела по работе с дебиторской задолженностью</p> </li> <li>

In [82]:
train[train['target']==1]['description'][1231]

'<strong>Обязанности:</strong> <ul> <li>Погрузка заявки (хлеба и хлебобулочных изделий) на автомобиль</li> <li>Развозка заявки по магазинам</li> </ul> <strong>Требования:</strong> <ul> <li>Стаж работы от 1 года</li> <li>Водительские права категории В,С</li> <li>Ответственность</li> <li>Аккуратность</li> <li>Исполнительность</li> </ul> <strong>Условия:</strong> <ul> <li>График работы 2/2 с 5-00 до 14-00</li> </ul>'

In [83]:
train[train['target']==1]['description'][1281]

'<p><strong>Обязанности:</strong></p> <ul> <li>работа по новым заявкам и с существующей клиентской базой на закрепленной территории;</li> <li>расширение существующей клиентской базы;</li> <li>проведение консультаций, презентаций;</li> <li>предоставление отчета.</li> </ul> <p><strong>Требования:</strong></p> <ul> <li>Ответственность;</li> <li>Грамотная речь;</li> <li>Готовность обучаться;</li> <li>Опыт работы с клиентами приветствуется.</li> </ul> <p> </p> <p><strong>Условия:</strong></p> <ul> <li>График работы 5/2 с 8 до 18 час.;</li> <li>Зарплата выплачивается еженедельно;</li> <li>Работа разъездного характера –с целью презентации;</li> <li>Дружный профессиональный коллектив;</li> <li>Возможность карьерного роста;</li> <li>Проводим бесплатное обучение;</li> <li>Оформление по ТК РФ.</li> </ul>'

In [93]:
test

,name,description
id,,
200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...
200005,Эксперт по обеспечению качества при сооружении...,<p><strong>Обязанности:</strong></p> <ul> <li>...
200006,Торговый представитель (Арзамас),<p><strong>Обязанности:</strong></p> <ul> <li>...
200007,Заместитель генерального директора по производ...,<p><strong>Обязанности:</strong></p> <ul> <li>...
200008,Backend Rust developer,<p><strong>Storiqa </strong>- это площадка для...


In [97]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

train_work = train.drop(['target', 'name', 'description'], axis=1)
test_work = test.drop(['name', 'description'], axis=1)
y = train['target']

In [103]:
test.index

Int64Index([200000, 200001, 200002, 200003, 200004, 200005, 200006, 200007,
            200008, 200009,
            ...
            370169, 370170, 370171, 370172, 370173, 370174, 370175, 370176,
            370177, 370178],
           dtype='int64', name='id', length=170179)

In [104]:
X_train, X_test, y_train, y_test = train_test_split(train_work, y, test_size=0.33, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])
result_predict = pd.DataFrame(data={'id': test.index, 'target': lr.predict_proba(test_work)[:,1]})
result_predict.to_csv('dsaponenko_06_predict.csv', index=False)